In [244]:
import pandas as pd
import numpy as np
import math
import random

In [212]:
bracket_df = pd.read_csv("C:/Users/Owner/OneDrive/Documents/cbb_survivor/cbb_survivor_sim/final_bracket_df.csv").iloc[:, 1:].copy()
bracket_df = bracket_df.loc[bracket_df["season"]==2025]

win_prob_df = pd.read_csv("C:/Users/Owner/OneDrive/Documents/cbb_survivor/cbb_survivor_sim/final_pairs_final.csv").iloc[:, 1:].copy()
win_prob_df = win_prob_df.loc[win_prob_df["season"]==2025]

In [407]:
bracket_df2["team1_name"].sort_values().unique()

array(['Akron', 'Alabama', 'Alabama St.', 'American', 'Arizona',
       'Arkansas', 'Auburn', 'BYU', 'Baylor', 'Bryant', 'Clemson',
       'Colorado St.', 'Connecticut', 'Creighton', 'Drake', 'Duke',
       'Florida', 'Georgia', 'Gonzaga', 'Grand Canyon', 'High Point',
       'Houston', 'Illinois', 'Iowa St.', 'Kansas', 'Kentucky', 'Liberty',
       'Lipscomb', 'Louisville', 'Marquette', 'Maryland', 'McNeese St.',
       'Memphis', 'Michigan', 'Michigan St.', 'Mississippi',
       'Mississippi St.', 'Missouri', 'Montana', "Mount St. Mary's",
       'Nebraska Omaha', 'New Mexico', 'Norfolk St.', 'North Carolina',
       'Oklahoma', 'Oregon', 'Purdue', 'Robert Morris',
       'SIU Edwardsville', "Saint Mary's", 'San Diego St.',
       'St. Francis PA', "St. John's", 'Tennessee', 'Texas', 'Texas A&M',
       'Texas Tech', 'Troy', 'UC San Diego', 'UCLA', 'UNC Wilmington',
       'Utah St.', 'VCU', 'Vanderbilt', 'Wisconsin', 'Wofford', 'Xavier',
       'Yale'], dtype=object)

In [404]:
win_prob_df["cur_team"].sort_values().unique()

array(['Akron', 'Alabama', 'Alabama St.', 'American', 'Arizona',
       'Arkansas', 'Auburn', 'BYU', 'Baylor', 'Bryant', 'Clemson',
       'Colorado St.', 'Connecticut', 'Creighton', 'Drake', 'Duke',
       'Florida', 'Georgia', 'Gonzaga', 'Grand Canyon', 'High Point',
       'Houston', 'Illinois', 'Iowa St.', 'Kansas', 'Kentucky', 'Liberty',
       'Lipscomb', 'Louisville', 'Marquette', 'Maryland', 'McNeese St.',
       'Memphis', 'Michigan', 'Michigan St.', 'Mississippi',
       'Mississippi St.', 'Missouri', 'Montana', "Mount St. Mary's",
       'Nebraska Omaha', 'New Mexico', 'Norfolk St.', 'North Carolina',
       'Oklahoma', 'Oregon', 'Purdue', 'Robert Morris',
       'SIU Edwardsville', 'Saint Francis', "Saint Mary's",
       'San Diego St.', "St. John's", 'Tennessee', 'Texas', 'Texas A&M',
       'Texas Tech', 'Troy', 'UC San Diego', 'UCLA', 'UNC Wilmington',
       'Utah St.', 'VCU', 'Vanderbilt', 'Wisconsin', 'Wofford', 'Xavier',
       'Yale'], dtype=object)

In [340]:
bracket_df_ff = bracket_df.loc[bracket_df["round"]=="First Four",["team1_name", "next_game_id1"]]
bracket_df_ff = bracket_df_ff.rename(columns = {"team1_name":"new_team", "next_game_id1":"next_game_id0"})
bracket_df_ff["join_term"] = "Winner"

ff_teams = bracket_df_ff["new_team"].unique()

bracket_join = bracket_df.copy()
bracket_join["join_term"] = bracket_join["team1_name"].str[:6]
bracket_df2 = bracket_join.merge(bracket_df_ff, on = ["next_game_id0","join_term"], how = "left")

mask = (bracket_df2["join_term"]=="Winner")
bracket_df2.loc[mask, "team1_name"] = bracket_df2.loc[mask, "new_team"]
bracket_df2.loc[mask, "team1_name"] = bracket_df2.loc[mask, "new_team"]
bracket_df2.loc[mask, "team1_name"] = bracket_df2.loc[mask, "new_team"]
bracket_df2 = bracket_df2.drop(["join_term","new_team"], axis=1)
bracket_df2 = bracket_df2.loc[bracket_df2["round"]=="Round of 64"]
bracket_df2 = bracket_df2.sort_values("team1_name")

In [383]:
bracket_df2

,round,team1_name,next_game_id0,next_game_id1,next_game_id2,next_game_id3,next_game_id4,season
62,Round of 64,Akron,E_R64_4,E_R32_2,E_S16_1,E_E8_1,F4_1,2025
27,Round of 64,Alabama,E_R64_8,E_R32_4,E_S16_2,E_E8_1,F4_1,2025
40,Round of 64,Alabama St.,S_R64_1,S_R32_1,S_S16_1,S_E8_1,F4_1,2025
58,Round of 64,American,E_R64_1,E_R32_1,E_S16_1,E_E8_1,F4_1,2025
23,Round of 64,Arizona,E_R64_4,E_R32_2,E_S16_1,E_E8_1,F4_1,2025
...,...,...,...,...,...,...,...,...
65,Round of 64,Vanderbilt,E_R64_7,E_R32_4,E_S16_2,E_E8_1,F4_1,2025
25,Round of 64,Wisconsin,E_R64_6,E_R32_3,E_S16_2,E_E8_1,F4_1,2025
75,Round of 64,Wofford,MW_R64_8,MW_R32_4,MW_S16_2,MW_E8_1,F4_2,2025
72,Round of 64,Xavier,MW_R64_5,MW_R32_3,MW_S16_2,MW_E8_1,F4_2,2025


In [384]:
ff_indices

[2, 3, 39, 43, 50, 51, 54, 66]

In [389]:
ff_pair_list

[[np.int64(2), np.int64(51)],
 [np.int64(3), np.int64(39)],
 [np.int64(43), np.int64(50)],
 [np.int64(54), np.int64(66)]]

In [349]:
ff_indices_df = bracket_df2["team1_name"].isin(ff_teams).reset_index()
ff_indices = ff_indices_df[ff_indices_df["team1_name"]==True].index.to_list()

bracket_df_ff_short = bracket_df_ff.sort_values("new_team")
bracket_df_ff_short["id"] = ff_indices

ff_game_ids = list(bracket_df_ff_short["next_game_id0"].unique())

ff_pair_list = []
for gid in ff_game_ids:
    pair = list(bracket_df_ff_short.loc[bracket_df_ff_short["next_game_id0"]==gid, "id"].unique())
    ff_pair_list.append(pair)


In [281]:
unique_games_r0 = bracket_df2["next_game_id0"].unique().tolist()
unique_games_r1 = bracket_df2["next_game_id1"].unique().tolist()
unique_games_r2 = bracket_df2["next_game_id2"].unique().tolist()
unique_games_r3 = bracket_df2["next_game_id3"].unique().tolist()
unique_games_r4 = bracket_df2["next_game_id4"].unique().tolist()

unique_game_ids = unique_games_r0 + unique_games_r1 + unique_games_r2 + unique_games_r3 + unique_games_r4
unique_game_ids.sort()

unique_game_nums = list(range(0,len(unique_game_ids)))

id_map = dict(zip(unique_game_ids, unique_game_nums))

new_bracket_df = bracket_df2.replace(id_map).sort_values("team1_name").drop(columns=["round", "team1_name", "season"])

unique_games_r0 = [id_map.get(x, x) for x in unique_games_r0]
unique_games_r1 = [id_map.get(x, x) for x in unique_games_r1]
unique_games_r2 = [id_map.get(x, x) for x in unique_games_r2]
unique_games_r3 = [id_map.get(x, x) for x in unique_games_r3]
unique_games_r4 = [id_map.get(x, x) for x in unique_games_r4]

bracket_arr_orig = new_bracket_df.to_numpy(dtype=float)

C:\Users\Owner\AppData\Local\Temp\ipykernel_11308\2909669786.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_bracket_df = bracket_df2.replace(id_map).sort_values("team1_name").drop(columns=["round", "team1_name", "season"])


In [395]:
# df = your dataframe
id_cols = [c for c in bracket_df2.columns if c.startswith("next_game_id")]
base_cols = ["team1_name"]  # add "season" here too if you want separate rows per season

long = (
    bracket_df2[base_cols + id_cols]
      .melt(id_vars=base_cols, value_vars=id_cols, value_name="game_id")
      .dropna(subset=["game_id"])
)

# wide: teams as rows, unique game_ids as columns, 1 if appears anywhere else 0
team_game_matrix = (
    pd.crosstab(long["team1_name"], long["game_id"])
      .astype(int)
).sort_values("team1_name")

game_match_arr_orig = team_game_matrix.to_numpy()


In [398]:
team_game_matrix.index

Index(['Akron', 'Alabama', 'Alabama St.', 'American', 'Arizona', 'Arkansas',
       'Auburn', 'BYU', 'Baylor', 'Bryant', 'Clemson', 'Colorado St.',
       'Connecticut', 'Creighton', 'Drake', 'Duke', 'Florida', 'Georgia',
       'Gonzaga', 'Grand Canyon', 'High Point', 'Houston', 'Illinois',
       'Iowa St.', 'Kansas', 'Kentucky', 'Liberty', 'Lipscomb', 'Louisville',
       'Marquette', 'Maryland', 'McNeese St.', 'Memphis', 'Michigan',
       'Michigan St.', 'Mississippi', 'Mississippi St.', 'Missouri', 'Montana',
       'Mount St. Mary's', 'Nebraska Omaha', 'New Mexico', 'Norfolk St.',
       'North Carolina', 'Oklahoma', 'Oregon', 'Purdue', 'Robert Morris',
       'SIU Edwardsville', 'Saint Mary's', 'San Diego St.', 'St. Francis PA',
       'St. John's', 'Tennessee', 'Texas', 'Texas A&M', 'Texas Tech', 'Troy',
       'UC San Diego', 'UCLA', 'UNC Wilmington', 'Utah St.', 'VCU',
       'Vanderbilt', 'Wisconsin', 'Wofford', 'Xavier', 'Yale'],
      dtype='object', name='team1_name')

In [280]:
df = win_prob_df.drop(columns=["season"], errors="ignore").sort_values("cur_team")

idx_cols = [c for c in df.columns if c not in ["opponent", "win_prob"]]

win_prob_wide = (
    df.pivot_table(
        index=idx_cols,
        columns="opponent",
        values="win_prob",
        aggfunc="first"
    )
    .reset_index()
).sort_values("cur_team").sort_index(axis=1).drop("cur_team", axis=1)


wp_arr_orig = np.nan_to_num(win_prob_wide.to_numpy(dtype=float))

In [287]:
still_alive_arr_orig = np.ones((68,1), dtype=int)


## Break

In [290]:
still_alive_arr_orig

array([[1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]])

In [219]:
bracket_arr_orig

array([[ 8.,  2., 13.,  0., 15.],
       [12.,  4., 14.,  0., 15.],
       [37., 33., 45., 32., 15.],
       [ 5.,  1., 13.,  0., 15.],
       [ 8.,  2., 13.,  0., 15.],
       [58., 51., 61., 47., 16.],
       [37., 33., 45., 32., 15.],
       [ 9.,  3., 14.,  0., 15.],
       [ 6.,  1., 13.,  0., 15.],
       [44., 36., 46., 32., 15.],
       [24., 19., 30., 17., 16.],
       [54., 49., 60., 47., 16.],
       [53., 48., 60., 47., 16.],
       [38., 33., 45., 32., 15.],
       [56., 50., 61., 47., 16.],
       [ 5.,  1., 13.,  0., 15.],
       [52., 48., 60., 47., 16.],
       [23., 18., 30., 17., 16.],
       [23., 18., 30., 17., 16.],
       [55., 49., 60., 47., 16.],
       [25., 19., 30., 17., 16.],
       [22., 18., 30., 17., 16.],
       [26., 20., 31., 17., 16.],
       [42., 35., 46., 32., 15.],
       [58., 51., 61., 47., 16.],
       [27., 20., 31., 17., 16.],
       [ 7.,  2., 13.,  0., 15.],
       [42., 35., 46., 32., 15.],
       [38., 33., 45., 32., 15.],
       [43., 3

In [262]:
game_match_arr_orig

array([[1, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(68, 62))

In [221]:
wp_arr_orig

array([[0.        , 0.07466937, 0.8470308 , ..., 0.62924956, 0.21865896,
        0.41046466],
       [0.92533063, 0.        , 0.98563628, ..., 0.95461298, 0.77618701,
        0.89613846],
       [0.1529692 , 0.01436372, 0.        , ..., 0.23460269, 0.04810819,
        0.11169479],
       ...,
       [0.37075044, 0.04538702, 0.76539731, ..., 0.        , 0.14154735,
        0.29089451],
       [0.78134104, 0.22381299, 0.95189181, ..., 0.85845265, 0.        ,
        0.71329753],
       [0.58953534, 0.10386154, 0.88830521, ..., 0.70910549, 0.28670247,
        0.        ]], shape=(68, 68))

## Break

In [423]:
wp_arr = wp_arr_orig.copy()
bracket_arr = bracket_arr_orig.copy()
game_match_arr = game_match_arr_orig.copy()
still_alive_arr = still_alive_arr_orig.copy()

In [ ]:
for g in ff_pair_list[0:4]:
    t1, t2 = g

    t1_wp = wp_arr[t1][t2]

    rand_num = random.random()

    if rand_num > t1_wp:
        still_alive_arr[t1] = 0
        game_match_arr[t1, :] = 0
    else:
        still_alive_arr[t2] = 0
        game_match_arr[t2, :] = 0

0.8413884095893586
0.8528304084545055
0.32177407462123186
0.48291531677541544


In [425]:
for g in unique_games_r0[0:60]:

    #print(np.flatnonzero(game_match_arr[:, g] == 1))
    t1, t2 = np.flatnonzero(game_match_arr[:, g] == 1)
    #print([t1,t2])

    t1_wp = wp_arr[t1][t2]
    #print(t1_wp)

    rand_num = random.random()
    #print(rand_num)


    if rand_num > t1_wp:
        still_alive_arr[t1] = 0
        game_match_arr[t1, :] = 0
    else:
        still_alive_arr[t2] = 0
        game_match_arr[t2, :] = 0


    


In [372]:
game_match_arr[:, 4]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0])

In [ ]:
brack